# 1. Unit Test

## 1.1 Unify Resolution


1. Given a Xarray with different variable with different resolution.

You need write a function, 
first, get the extent of input ds, and resize(resample) differnet variables into same size with given resolution.


Parameter: (ds,new_lon_resolution, new_lat_resolution, high_low_method, low_high_method)




In [ ]:
import xarray as xr
import numpy as np
def resample_dataset(ds, new_lon_resolution, new_lat_resolution, high_low_method, low_high_method):
    # Step 1: Determine the extent of the input dataset
    lon_min, lon_max = ds.longitude.min(), ds.longitude.max()
    lat_min, lat_max = ds.latitude.min(), ds.latitude.max()

    # Step 2: Create new longitude and latitude coordinates
    new_lon = np.arange(lon_min, lon_max, new_lon_resolution)
    new_lat = np.arange(lat_min, lat_max, new_lat_resolution)

    # Step 3: Resample each variable
    resampled_ds = xr.Dataset()
    for var in ds.data_vars:
        # Determine the current resolution
        current_resolution = (ds[var].longitude[1] - ds[var].longitude[0], ds[var].latitude[1] - ds[var].latitude[0])

        # Choose the appropriate resampling method
        if current_resolution < (new_lon_resolution, new_lat_resolution):
            method = low_high_method
        else:
            method = high_low_method

        # Resample the variable
        resampled_var = ds[var].interp(longitude=new_lon, latitude=new_lat, method=method)
        resampled_ds[var] = resampled_var

    return resampled_ds



In [21]:
#dataset.to_netcdf("C:/Users/isxzl/OneDrive/Code/AutoGeo/data/saved_on_disk.nc")
dataset = xr.open_dataset("C:/Users/isxzl/OneDrive/Code/AutoGeo/data/saved_on_disk.nc")
new_lon_resolution = 0.1
new_lat_resolution = 0.1
resampled_ds = resample_dataset(dataset, new_lon_resolution, new_lat_resolution, 'linear', 'linear')
 


## 1.2 Create Toy Data_lon)

In [3]:
import sys 
sys.path.append("C:/Users/isxzl/OneDrive/Code/AutoGeo")
from sample.dataset import create_xarray_within_boundary
import pandas as pd

# Example usage
boundary = (27, 118, 31.5, 122.5)  # min_latitude, min_longitude, max_latitude, max_longitude
resolution = 0.025  # 0.1 degree
datetime_list = [pd.Timestamp('2021-10-14 09:00:00'), pd.Timestamp('2021-10-15 09:00:00'), pd.Timestamp('2021-10-16 09:00:00')]  # List of datetime objects

ds = create_xarray_within_boundary(datetime_list, boundary, resolution)
ds


<xarray.Dataset>
Dimensions:               (Datetime: 3, NLatitude: 180, ELongtitude: 180)
Coordinates:
  * Datetime              (Datetime) datetime64[ns] 2021-10-14T09:00:00 ... 2...
  * NLatitude             (NLatitude) float64 27.0 27.02 27.05 ... 31.45 31.47
  * ELongtitude           (ELongtitude) float64 118.0 118.0 ... 122.5 122.5
Data variables:
    Negative_oxygen_ions  (Datetime, NLatitude, ELongtitude) float64 0.0 ... 0.0

## 1.3 Merge feature Xarr to Label Xrray

In [6]:
import sys 
sys.path.append("C:/Users/isxzl/OneDrive/Code/AutoGeo")
from sample.dataset import merge_features_with_progress
import xarray as xr
import pandas as pd
import numpy as np 

# Creating dummy xarray Datasets
times = pd.date_range('2021-01-01', periods=3)
lons = np.linspace(120.0, 121.0, 3)
lats = np.linspace(30.0, 31.0, 3)

# Dataset A
ds_a = xr.Dataset(
    {
        "temperature": (("Datetime", "ELongtitude", "NLatitude"), np.random.rand(3, 3, 3)),
    },
    coords={
        "Datetime": times,
        "ELongtitude": lons,
        "NLatitude": lats,
    },
)

# Dataset B with features
ds_b = xr.Dataset(
    {
        "z": (("time", "longitude", "latitude"), np.random.rand(3, 3, 3)),
        "sp": (("time", "longitude", "latitude"), np.random.rand(3, 3, 3)),
    },
    coords={
        "time": times,
        "longitude": lons,
        "latitude": lats,
    },
)

features_list = ['z', 'sp']

# Test function
ds_updated = merge_features_with_progress(ds_a, ds_b, features_list)


Merging Features: 100%|███████████████████████| 2/2 [00:00<00:00, 42.12it/s]


# 2. Paper Code

## 2.1 Make Xarray Prediction Dataset

Given:  
- a lot of path and dir.
- resolution, range, date

Wanted: 
- A unified Xarray 

In [ ]:
start_time='2022-01-01 00:00:00'
end_time='2022-03-31 23:00:00'
freq='4H'
grib_fea_path=r"C:\Datasets\Zhejiang20-23RS\ERA5_featureRanking1\2022_Jan_2_Mar_feature.nc"
ear5_dir = r'C:\Datasets\Zhejiang20-23RS\ERA5_featureRanking1\air'
ear5_l_dir = r'C:\Datasets\Zhejiang20-23RS\ERA5_featureRanking1\land'
srtm_path=r"C:\Datasets\Zhejiang20-23RS\Common_Dataset\DEM_SRTM/2022001.img"
landuse_dir= r'C:\Datasets\Zhejiang20-23RS\Common_Dataset\LandUse'

In [ ]:
start_time='2022-01-01 00:00:00'
end_time='2022-03-31 23:00:00'
freq='4H'
grib_fea_path=r"C:\Datasets\Zhejiang20-23RS\ERA5_ThreeYear\2020_2_2023_Feature.nc"
ear5_dir = r'C:\Datasets\Zhejiang20-23RS\ERA5_ThreeYear\air'
ear5_l_dir = r'C:\Datasets\Zhejiang20-23RS\ERA5_ThreeYear\land'
srtm_path=r"C:\Datasets\Zhejiang20-23RS\Common_Dataset\DEM_SRTM/2022001.img"
landuse_dir= r'C:\Datasets\Zhejiang20-23RS\Common_Dataset\LandUse'

In [10]:
import sys 
sys.path.append("C:/Users/isxzl/OneDrive/Code/AutoGeo")
from sample.dataset import create_xarray_within_boundary
import pandas as pd

# Example usage
boundary = (27, 118, 31.5, 122.5)  # min_latitude, min_longitude, max_latitude, max_longitude
resolution = 0.025  # 0.1 degree
# Generate a list of datetime objects from the start of 2022 to the end of March 2022 with hourly resolution
datetime_list = pd.date_range(start=start_time, end=end_time, freq=freq).tolist()


ds = create_xarray_within_boundary(datetime_list, boundary, resolution)

In [3]:
import joblib
import os
import sys
sys.path.append("C:/Users/isxzl/OneDrive/Code/AutoGeo")
from sample.grib import load_era5_batch
from sample.geotiff import load_srtm
from sample.hdf import load_modis_batch
from sample.dataset import merge_features_with_progress,merge_features_with_broadcast
from sample.unit_test import get_xarray_memory_usage_gb


print("era5_air") 
ds_temp=load_era5_batch(ear5_dir)
ds = merge_features_with_progress(ds, ds_temp,memory_optim=True)
print(f"Total memory usage of the {'era5'}: {get_xarray_memory_usage_gb(ds_temp)} GB")
print(f"Total memory usage of the {'ds'}: {get_xarray_memory_usage_gb(ds)} GB")


print("era5_land") 
ds_temp=load_era5_batch(ear5_l_dir)
ds = merge_features_with_progress(ds, ds_temp,memory_optim=True)
print(f"Total memory usage of the {'era5'}: {get_xarray_memory_usage_gb(ds_temp)} GB")
print(f"Total memory usage of the {'ds'}: {get_xarray_memory_usage_gb(ds)} GB")


print("srtm") 
ds_temp=load_srtm(srtm_path)
ds = merge_features_with_broadcast(ds, ds_temp)
print(f"Total memory usage of the {'srtm'}: {get_xarray_memory_usage_gb(ds_temp)} GB")
print(f"Total memory usage of the {'ds'}: {get_xarray_memory_usage_gb(ds)} GB")

print("landuse") 
ds_temp=load_modis_batch(landuse_dir)   
ds = merge_features_with_broadcast(ds, ds_temp,method="nearest")
print(f"Total memory usage of the {'landuse'}: {get_xarray_memory_usage_gb(ds_temp)} GB")
print(f"Total memory usage of the {'ds'}: {get_xarray_memory_usage_gb(ds)} GB")


# Test function
ds.to_netcdf(grib_dir, format='NETCDF4')
#ds = xr.open_dataset(r"C:\Datasets\Zhejiang20-23RS\ERA5_featureRanking1\2022_Jan_2_Mar.nc")
print(f"save to {grib_dir}")

era5_air
[autoGEO][Info] Process 1th file in 3
========filter file detected========
========merged nc file detected========
[autoGEO][Info] Process 2th file in 3
========filter file detected========
========merged nc file detected========
[autoGEO][Info] Process 3th file in 3
========filter file detected========
========merged nc file detected========


Merging Features: 100%|████████████████████████████████████████████████████████████████| 86/86 [02:06<00:00,  1.48s/it]


Total memory usage of the era5: 0.24287626147270203 GB
Total memory usage of the ds: 11.340916156768799 GB
era5_land
[autoGEO][Info] Process 1th file in 14
========filter file detected========
========merged nc file detected========
[autoGEO][Info] Process 2th file in 14
========filter file detected========
========merged nc file detected========
[autoGEO][Info] Process 3th file in 14
========filter file detected========
========merged nc file detected========
[autoGEO][Info] Process 4th file in 14
========filter file detected========
========merged nc file detected========
[autoGEO][Info] Process 5th file in 14
========filter file detected========
========merged nc file detected========
[autoGEO][Info] Process 6th file in 14
========filter file detected========
========merged nc file detected========
[autoGEO][Info] Process 7th file in 14
========filter file detected========
========merged nc file detected========
[autoGEO][Info] Process 8th file in 14
========filter file detected====

Merging Features: 100%|████████████████████████████████████████████████████████████████| 44/44 [01:13<00:00,  1.68s/it]


Total memory usage of the era5: 0.7325248718261719 GB
Total memory usage of the ds: 16.42477512359619 GB
srtm


Merging Features: 100%|██████████████████████████████████████████████████████████████████| 1/1 [00:01<00:00,  1.20s/it]


Total memory usage of the srtm: 0.21725893020629883 GB
Total memory usage of the ds: 16.55513048171997 GB
landuse
Load merged file


Merging Features: 100%|██████████████████████████████████████████████████████████████████| 1/1 [00:01<00:00,  1.60s/it]


Total memory usage of the landuse: 2.270999640226364 GB
Total memory usage of the ds: 16.68548583984375 GB


## 2.2 Make Tabular

Given:
- a Xarray Data
- DF


In [ ]:
start_date = '2022-01-01 00:00:00'
end_date = '2022-03-31 23:00:00'
freq='4H'


grib_fea_path=r"C:\Datasets\Zhejiang20-23RS\ERA5_featureRanking1\2022_Jan_2_Mar_feature.nc"
site_path = r'C:\Datasets\Zhejiang20-23RS\site_withCoor.csv'


site_Feature_pkl= r'C:\Datasets\Zhejiang20-23RS\ERA5_featureRanking1\site_withCoor_withFea.pkl'
site_Feature_clean_pkl= r'C:\Datasets\Zhejiang20-23RS\ERA5_featureRanking1\site_withCoor_withFea_cleaned.pkl'

In [1]:
import pandas as pd
import sys 
sys.path.append("C:/Users/isxzl/OneDrive/Code/AutoGeo")
from sample.dataset import filter_dataframe_by_date
from sample.retrieval import Xarray2Tabular
import xarray as xr
import pandas as pd
import numpy as np
 
df=filter_dataframe_by_date(csv_path,start_date,end_date)
ds = xr.open_dataset(grib)
  
df = Xarray2Tabular(ds,df)
df.to_pickle(site_Feature_pkl)    #to save the dataframe, df to 123.pkl



C:\Users\isxzl\anaconda3\envs\EO\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Processing sites: 100%|████████████████████████████████████████████████████████████████| 60/60 [14:54<00:00, 14.90s/it]


In [5]:
import pandas as pd
import numpy as np


df = pd.read_pickle(site_Feature_pkl) #to load 123.pkl back to the dataframe df
 
# Define the datetime list
datetime_list = pd.date_range(start=start_date , end=end_date, freq=freq).tolist() 
df_filtered = df[df['time'].isin(datetime_list)]

for i in range(40,45):
    # Count the non-NaN values for each row in the filtered DataFrame
    non_nan_counts = df_filtered.apply(lambda row: row.count(), axis=1)
    rows_to_delete = non_nan_counts[non_nan_counts <= i].index
    print(f"Number of rows with <={i} non-NaN columns that were deleted: {len(rows_to_delete)} of {len(df_filtered)}")
df_final = df_filtered.drop(rows_to_delete)
print(df_final)
df_final.to_pickle(site_Feature_clean_pkl)

Number of rows with <=40 non-NaN columns that were deleted: 969 of 41380
Number of rows with <=41 non-NaN columns that were deleted: 969 of 41380
Number of rows with <=42 non-NaN columns that were deleted: 969 of 41380
Number of rows with <=43 non-NaN columns that were deleted: 969 of 41380
Number of rows with <=44 non-NaN columns that were deleted: 969 of 41380


## 2.3 Pseudo Tabular

In [ ]:
start_date = '2022-01-01 00:00:00'
end_date = '2022-03-31 23:00:00'
freq='4H'


grib_fea_path=r"C:\Datasets\Zhejiang20-23RS\ERA5_featureRanking1\2022_Jan_2_Mar_feature.nc"
site_path = r'C:\Datasets\Zhejiang20-23RS\site_withCoor.csv'
pseudo_site_path = r'C:\Datasets\Zhejiang20-23RS\ERA5_featureRanking1\pseudosite_withCoor_FeaImportance.pkl'

site_Feature_pkl= r'C:\Datasets\Zhejiang20-23RS\ERA5_featureRanking1\pseudosite_withCoor_withFea.pkl'
site_Feature_clean_pkl= r'C:\Datasets\Zhejiang20-23RS\ERA5_featureRanking1\pseudosite_withCoor_withFea_cleaned.pkl'


In [4]:
import pandas as pd
import sys 
sys.path.append("C:/Users/isxzl/OneDrive/Code/AutoGeo")
from sample.dataset import filter_dataframe_by_date,pseudo_df_generator
from sample.retrieval import Xarray2Tabular
import xarray as xr
import pandas as pd
import numpy as np

 
df=filter_dataframe_by_date(site_path,start_date,end_date)
ds = xr.open_dataset(grib_fea_path)


df=pseudo_df_generator(df)
df.to_pickle(pseudo_site_path) 
df=pd.read_pickle(pseudo_site_path)


In [5]:

df = Xarray2Tabular(ds,df)
df.to_pickle(site_Feature_pkl)   

Processing Site_number=0: 100%|████████████████████████████████████████████████| 61050/61050 [5:30:13<00:00,  3.08it/s]


In [ ]:
df[df["Site_number"]==0]

In [7]:
 
datetime_list = pd.date_range(start=start_date , end=end_date, freq=freq).tolist() 
df_filtered = df[df['time'].isin(datetime_list)]

for i in range(40,45): 
    # Count the non-NaN values for each row in the filtered DataFrame
    non_nan_counts = df_filtered.apply(lambda row: row.count(), axis=1)
    rows_to_delete = non_nan_counts[non_nan_counts <= i].index
    print(f"Number of rows with <={i} non-NaN columns that were deleted: {len(rows_to_delete)} of {len(df_filtered)}")
df_final = df_filtered.drop(rows_to_delete)
print(df_final)
df_final.to_pickle(site_Feature_clean_pkl)

Number of rows with <=40 non-NaN columns that were deleted: 969 of 41380
Number of rows with <=41 non-NaN columns that were deleted: 969 of 41380
Number of rows with <=42 non-NaN columns that were deleted: 969 of 41380
Number of rows with <=43 non-NaN columns that were deleted: 969 of 41380
Number of rows with <=44 non-NaN columns that were deleted: 969 of 41380
       Site_number     Site_name                 time   longitude   latitude  \
0            58448         临安气象站  2022-01-01 00:00:00       119.7      30.21   
4            58448         临安气象站  2022-01-01 04:00:00       119.7      30.21   
8            58448         临安气象站  2022-01-01 08:00:00       119.7      30.21   
12           58448         临安气象站  2022-01-01 12:00:00       119.7      30.21   
16           58448         临安气象站  2022-01-01 16:00:00       119.7      30.21   
...            ...           ...                  ...         ...        ...   
166086           0  PseudoPoints  2022-03-17 20:00:00   119.81195  28.70776

## 2.4 DF merged to Xarray

In [26]:

import pandas as pd
import sys 
sys.path.append("C:/Users/isxzl/OneDrive/Code/AutoGeo")
from sample.dataset import filter_dataframe_by_date
from sample.retrieval import Xarray2Tabular
import xarray as xr
 


start_date = '2022-01-01'
end_date = '2022-03-31'
grib=r"C:\Datasets\Zhejiang20-23RS\ERA5_featureRanking1\2022_Jan_2_Mar.nc"
csv_path = r'C:\Datasets\Zhejiang20-23RS\merged_coor.csv'
csv_pkl = r'C:\Datasets\Zhejiang20-23RS\merged_coor.pkl'

df=filter_dataframe_by_date(csv_path,start_date,end_date)
ds = xr.open_dataset(grib)
 
asasf
ds = Tabular2array (ds, df)

Givn THIS CODE, MAKEA function Xarray2Tabular.

df is a pd frame which has Site_name, Site_number, time longitude, latitude, Negative_oxygen_ions etc,.
ds has a lots of varibles, but with three dimnesion longitude, latitude, time
You need to 
1. drop Negative_oxygen_ions of ds if ds has.
1., group df by Site_name, they will share same longitude, latitude but with differnet time.

Then, for each site, we have longitude and latitude, CALCULATE ITS NEAREST lon and latitude in ds.
1. create an empty list(could be ds, or np or df or series) with same length of time (in ds).
2. intreparate (align) site_csv to it, for other keep nan.
2. update the subdataset list to ds.
 

SyntaxError: invalid syntax (4120921383.py, line 22)

In [ ]:
import pandas as pd
import xarray as xr
from scipy.spatial import cKDTree
import numpy as np

def update_ds_with_df(ds: xr.Dataset, df: pd.DataFrame) -> xr.Dataset:
    # Drop 'Negative_oxygen_ions' from ds if it exists
    if 'Negative_oxygen_ions' in ds:
        ds = ds.drop_vars('Negative_oxygen_ions')
    
    # Assuming ds has dimensions time, latitude, and longitude named exactly like that.
    # Convert the times in ds to a pandas datetime format for comparison
    ds['time'] = pd.to_datetime(ds['time'].values)
    
    # Prepare KDTree for spatial lookup using ds coordinates
    lon_lat_pairs = np.vstack([ds['longitude'].values.ravel(), ds['latitude'].values.ravel()]).T
    tree = cKDTree(lon_lat_pairs)

    # Iterate over each row in df to find the nearest point in ds and update values accordingly
    for index, row in df.iterrows():
        # Find nearest spatial point in ds
        dist, pos = tree.query([row['longitude'], row['latitude']], k=1)
        nearest_lon, nearest_lat = lon_lat_pairs[pos]

        # Find nearest time point in ds
        nearest_time = ds.sel(time=row['time'], method='nearest').time.values
        
        # Assuming tolerance checks for time (6 hours) and space (0.1 degrees)
        if abs(ds['longitude'].sel(longitude=nearest_lon) - row['longitude']) <= 0.1 and \
           abs(ds['latitude'].sel(latitude=nearest_lat) - row['latitude']) <= 0.1 and \
           abs(pd.to_datetime(nearest_time) - pd.to_datetime(row['time'])) <= pd.Timedelta('6H'):
            # Update ds with Negative_oxygen_ions value from df
            ds['Negative_oxygen_ions'].loc[dict(time=nearest_time, longitude=nearest_lon, latitude=nearest_lat)] = row['Negative_oxygen_ions']

    return ds
